In [1]:
freq_dict = {'15-min':'15T',
            '1-hour':'1H',
            '2-hour':'2H',
            '4-hour':'4H',
            '6-hour':'6H',
            '8-hour':'8H',
            '12-hour':'12H',
            '1-day':'1D',
            '2-day':'2D',
            '7-day':'7D',
            '15-day':'15D',
            '1-month':'1M'}

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy



In [4]:
metadata_df = pd.read_csv("../metadata/metadata.csv",index_col=0)

In [19]:
city = 'Austin'

In [20]:
st = pd.HDFStore(os.path.expanduser("/Users/xox/monthly-12-17.h5"), mode='r')

In [21]:
freq='1M'

In [22]:
city_data = metadata_df[metadata_df['city'] == city]
city_homes = city_data.index.values.astype('int')

In [23]:
len(city_homes)

902

In [51]:
homes = {}
count = 0
for freq in ['1M']:
    print(freq)
    freq_num = int(freq[:-1])
    out = []
    homes[freq] = []
    city_data = metadata_df[metadata_df['city'] == city]
    city_homes = city_data.index.values.astype('int')
    for home in city_homes:
#         print(home)
        try: 
            data_df = st['/%d' %home]
            count += 1
            homes[home] = data_df.resample(freq).mean()
        except Exception as e:
            continue


1M


In [69]:
home_ids = homes.keys()

In [119]:
home_readings = {}
for year in range(2012, 2018):
    home_readings[year] = {}
for hid in home_ids:
    for year in range(2012, 2018):
        try:
            year_readings = homes[hid][str(year)]
            if len(year_readings) < 12:
                continue
            for m in range(12):
                old_index = year_readings.index[m]
                new_index = str(old_index)[:7]
                year_readings = year_readings.rename(index={old_index:new_index})
                home_readings[year][hid] = year_readings
                
        except:
            continue

In [141]:
# with all observed aggregate readings
all_aggregate = {}
for year in range(2013, 2018):
    all_aggregate[year] = {}
    for hid in home_readings[year].keys():
        try:
            if home_readings[year][hid]['use'].isnull().sum() == 0:
                all_aggregate[year][hid] = home_readings[year][hid]
        except Exception as e:
            continue
for year in range(2013, 2018):
    print(year, len(all_aggregate[year].keys()))

2013 160
2014 322
2015 307
2016 260
2017 197


In [142]:
year = 2013

In [152]:
app_nan = all_aggregate[year][9121].isnull().sum()
print(app_nan)

use        0
air1       0
drye1     12
range1     0
dtype: int64


In [159]:
app_count = {}
for year in range(2013, 2018):
    app_count[year] = {}
    for hid in all_aggregate[year].keys():
        app_nan = all_aggregate[year][hid].isnull().sum()
        for app in app_nan.keys():
            if app_nan[app] == 0:
                if app in app_count[year]:
                    app_count[year][app] += 1
                else:
                    app_count[year][app] = 1

In [487]:
df = pd.DataFrame(app_count)

In [489]:
df[(df > 100).any(1)]

,2013,2014,2015,2016,2017
air1,91.0,239.0,274.0,235.0,175.0
clotheswasher1,44.0,118.0,156.0,131.0,85.0
dishwasher1,48.0,181.0,171.0,143.0,93.0
drye1,46.0,113.0,158.0,135.0,103.0
furnace1,64.0,215.0,210.0,179.0,119.0
kitchenapp1,28.0,140.0,135.0,110.0,73.0
microwave1,35.0,116.0,148.0,125.0,77.0
refrigerator1,57.0,195.0,196.0,159.0,104.0
use,160.0,322.0,307.0,260.0,197.0


In [488]:
candidate_app = df[(df > 30).all(1)].index.tolist()

In [490]:
candidate_app = ['air1', 'clotheswasher1', 'dishwasher1', 'drye1', 'furnace1', 'kitchenapp1', 'microwave1', 'refrigerator1', 'use']

In [491]:
c = 0
overall = 0
cand = {}
for year in range(2013, 2018):
    cand[year] = {}
    for hid in all_aggregate[year].keys():
        if all_aggregate[year][hid].isnull().sum()[candidate_app].sum() == 0:
#             print(year, hid)
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(candidate_app)]
            if len(candidate.columns) == len(candidate_app):
                cand[year][hid] = candidate
    print(year, len(cand[year].keys()))
    overall += len(cand[year].keys())
print('overall', overall)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


2013 10
2014 34
2015 54
2016 41
2017 31
overall 170


In [492]:
def get_dataset(selected_years, selected_appliance, all_observed, num_columns=1):
    data = {}
    for year in selected_years:
        data[year] = {}
        for hid in all_aggregate[year].keys():
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(selected_appliance)]
            if all_observed:
                if len(candidate.columns) == len(selected_appliance) and candidate.isnull().sum().sum() == 0:
                    data[year][hid] = candidate
            else:
                if len(candidate.columns) >= num_columns:
                    data[year][hid] = candidate

    return data
            

In [493]:
def get_info(data):
    overall = 0
    for year in data.keys():
        print(year, len(data[year].keys()))
        overall += len(data[year].keys())
    print("overall: ", overall)

In [504]:
selected_years = [2013, 2014, 2015, 2016, 2017]
selected_appliance = ['air1', 'clotheswasher1', 'dishwasher1', 'furnace1',  'microwave1', 'refrigerator1', 'use']
all_observed = True

In [505]:
data = get_dataset(selected_years, selected_appliance, all_observed, 2)
get_info(data)

2016 85
2017 53
2013 21
2014 59
2015 101
overall:  319


In [503]:
np.save("./data-2013-2017-observed.npy", data)